### **Import**

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import re

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Writing multiple dataframes to worksheets using Pandas and XlsxWriter
# !pip install xlsxwriter
import xlsxwriter
import glob

## **Выгрузка**

In [2]:
xls = pd.ExcelFile(r'C:\Users\u1-510\Desktop\Facebook\Статистика аккаунтов.xlsx')
fbtool = pd.read_excel(xls, 'Sheet1')
crm = pd.read_excel(xls, 'Sheet2')

### **Статистика предыдущий выгрузки**

In [3]:
# xls1= pd.ExcelFile(r'C:\Users\u1-510\Desktop\Facebook\OLD\UZ_Test.xlsx')
# df1 = pd.read_excel(xls1, 'Information')

In [4]:
# Должен иметь шапку, первую строку "Adset" в файле не удалять 
df_stop = pd.read_excel(r'C:\Users\u1-510\Desktop\Facebook\Status.xlsx', names=['bm'], header=None)

### **Настройки**

In [5]:
# Время выгрузки
time_unloading = '18:30'
# Регион
GEO = 'DE'
# Название файла
file_name = 'DE_Test.xlsx'

# **Pivot table**

## **Fbtool**

In [6]:
# Изменение header
fbtool.columns = fbtool.iloc[0]
fbtool = fbtool.reindex(fbtool.index.drop(0))
cols = list(fbtool.columns)
cols[1] = 'Utm_campaign'
cols[2] = 'Сabinet'
cols[3] = 'Cost'
cols[4] = 'Show'
cols[5] = 'Link_cliks'
cols[6] = 'Install'
fbtool.columns = cols

In [7]:
# Удаление NaN
fbtool = fbtool.replace(0,np.nan).dropna(axis=1,how="all")

In [8]:
# Удаление колонок
fbtool.drop(["Сabinet", "CPI", "CPC", "CPM", "CTR"], axis=1, inplace=True )

In [9]:
# удалить данные после пробела
fbtool['Utm_campaign'] = fbtool['Utm_campaign'].map(lambda x: x.split()[0])

In [10]:
#print(fbtool['Utm_campaign'].values)

In [11]:
# текст по столбцам
#fb_addset = fbtool['Utm_campaign'].str.split('_', expand=True)
fb_addset = fbtool['Utm_campaign'].str.split(r'\_|\-', expand=True)

In [12]:
# Header for adset
fb_addset.columns=['adset', 'geo', 'brand', 'page', 'soc', 
                   'bm', 'acc', 'camp', 'adset1', 'int', 'placement', 
                   'gender', 'age', 'text', 'creo', 'app', 'os', 'mb']

In [13]:
# соединить два датафрейма по столбцам
fbtool_final = pd.concat([fbtool,fb_addset],axis=1)

In [14]:
# Время
fbtool_final['Time'] = time_unloading

In [15]:
# # ACC только V
# fbtool_final = fbtool_final.loc[fbtool_final.acc == ACC]

In [16]:
# Регион
fbtool_final = fbtool_final.loc[fbtool_final.geo == GEO]

In [17]:
# Замена NaN
fbtool_final.fillna(0, inplace=True)

In [18]:
# Изменение форматов
fbtool_final['Cost'] = fbtool_final['Cost'].astype(float)
fbtool_final['Show'] = fbtool_final['Show'].astype(int)
fbtool_final['Link_cliks'] = fbtool_final['Link_cliks'].astype(int)
fbtool_final['Install'] = fbtool_final['Install'].astype(int)

In [19]:
fbtool_final = fbtool_final.loc[fbtool_final.geo == GEO]

## **CRM**

In [20]:
# Удаление колонок
crm.drop(["Utm_source", 'Utm_content', "Utm_medium"], axis=1, inplace=True)

In [21]:
# Переименование колонок
crm = crm.rename(columns=
                       {'Registration count': 'Reg',
                        'Ftd count': 'Ftd'
                       })

In [22]:
# замена прочерков
crm['Ftd'].replace('-', 0, inplace=True)

In [23]:
# Изменение форматов
crm['Reg'] = crm['Reg'].astype(int)
crm['Ftd'] = crm['Ftd'].astype(int)

### **Combining**

In [24]:
# Left join
df = fbtool_final.merge(crm, how='left', on=["Utm_campaign"])

In [25]:
# Замена NaN
df.fillna(0, inplace=True)

In [26]:
# Изменение форматов
df['Reg'] = df['Reg'].astype(int)

## **GROUP BY**

In [27]:
# Группировка при выборке
df_group = pd.DataFrame(df.groupby(['bm', 'Utm_campaign'], as_index=False).sum())

In [28]:
# Вычисление
df_group = df_group.assign(Instal_cost = df_group.Cost/df_group.Install)
df_group = df_group.assign(Reg_cost = df_group.Cost/df_group.Reg)
df_group = df_group.assign(CTR = df_group.Link_cliks/df_group.Show*100)
df_group = df_group.assign(FTD_cost = df_group.Cost/df_group.Ftd)

In [29]:
# Удаление колонок
df_group.drop(['Show', 'Link_cliks'], axis=1, inplace=True)

In [30]:
# Замена Inf
df_group.replace([np.inf, -np.inf], 0, inplace=True)

In [31]:
# Словарь класификации
codes_cats = df_group.Utm_campaign.astype('category')
dict_cats = dict(enumerate(codes_cats.cat.categories))

In [32]:
# Присвоенные cat.codes
df_group['ID'] = df_group.Utm_campaign.astype('category').cat.codes

# Розшифровка
#df_group['level_back'] = df_group['ID'].map(dict_cats)

## **Modeling**

In [33]:
# machine learning
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score , accuracy_score, precision_score
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier

In [34]:
df_train = pd.read_excel(r'C:\Users\u1-510\Desktop\Facebook\Model\030FA000.xlsx')

In [35]:
df_test = df_group[['ID', 'Cost', 'Install', 'Reg', 'Instal_cost', 'Reg_cost']].copy()

In [36]:
# Переименование и удаление колонок
# df_train.drop(['Ftd', 'FTD_cost'], axis=1, inplace=True)

df_train = df_train.rename(columns=
                       {'Name': 'ID',})

In [37]:
X = df_train.drop("Target",axis=1)
y = df_train["Target"]

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [39]:
#Using kfold cross validation
from sklearn.model_selection import KFold, cross_val_predict
kf = KFold(shuffle=True, random_state=42, n_splits=3)
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = (X.iloc[train_index, :], 
                                        X.iloc[test_index, :], 
                                        y[train_index], 
                                        y[test_index])

In [40]:
from xgboost.sklearn import XGBClassifier
xgb=XGBClassifier(learning_rate=0.001, objective='multi:softprob', n_estimators=2500,
                                max_depth=4, min_child_weight=0,
                                gamma=0, subsample=0.7,
                                colsample_bytree=0.7,
                                seed=27,
                                reg_alpha=0.00006,
                                use_label_encoder=False)
xgb.fit(X_train,y_train)
predict_12=xgb.predict(X_test)

[15:55:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [41]:
print(accuracy_score(y_test,predict_12))
print(confusion_matrix(y_test,predict_12))
print(classification_report(y_test,predict_12))

0.9776357827476039
[[ 93   4   0]
 [  0  60   1]
 [  2   0 153]]
              precision    recall  f1-score   support

           0       0.98      0.96      0.97        97
           1       0.94      0.98      0.96        61
           2       0.99      0.99      0.99       155

    accuracy                           0.98       313
   macro avg       0.97      0.98      0.97       313
weighted avg       0.98      0.98      0.98       313



In [42]:
pred = xgb.predict(df_test)
pred = pred.astype(int)

## **Analysis**

### **Predict page**

In [43]:
try:
  submission = pd.DataFrame({'ID':df_test['ID'],'score':pred})

  predict = pd.merge(df_group, 
                    submission,
                    on = 'ID')

  def score_df(el):
    if el == 0:
      return 'Restrat'
    elif el == 1:
      return 'Active'
    else:
      return 'Threshold'

  predict['Score'] = predict['score'].map(score_df)

  stop_list = df_stop['bm'].tolist()
  def stop_df(el):
    if el in stop_list:
      return 'Stop'
    else:
      return 'Active'

  predict['Status'] = predict['bm'].map(stop_df)

  predict = predict[['bm', 'Utm_campaign', 'Cost', 'Install', 'Reg', 
                'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR', 'Score', 'Status']]

  # Два знака после запятой
  predict = predict.round(decimals=2)

  # Замена Inf
  predict.replace(np.nan, 0, inplace=True)

  # Сортировка Cost по возростанию
  predict = predict.sort_values(['Status', 'Score', 'Cost'], ascending=[True, True, False])

  # Замена 0 на -
  predict['Instal_cost'].replace(0, '-', inplace=True)
  predict['Reg_cost'].replace(0, '-', inplace=True)
  predict['FTD_cost'].replace(0, '-', inplace=True)
  
  print(1)
except:
  submission = pd.DataFrame({'ID':df_test['ID'],'score':pred})

  predict = pd.merge(df_group, 
                    submission,
                    on = 'ID')

  def score_df(el):
    if el == 0:
      return 'Restrat'
    elif el == 1:
      return 'Active'
    else:
      return 'Threshold'

  predict['Score'] = predict['score'].map(score_df)

  predict = predict[['bm', 'Utm_campaign', 'Cost', 'Install', 'Reg', 
                'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR', 'Score']]

  # Два знака после запятой
  predict = predict.round(decimals=2)

  # Замена Inf
  predict.replace(np.nan, 0, inplace=True)

  # Сортировка Cost по возростанию
  predict = predict.sort_values(['Score', 'Cost'], ascending=[True, False])

  # Замена 0 на -
  predict['Instal_cost'].replace(0, '-', inplace=True)
  predict['Reg_cost'].replace(0, '-', inplace=True)
  predict['FTD_cost'].replace(0, '-', inplace=True)
  print(2)

1


### **Total page**

In [44]:
try:
  df = df[['Time', 'Utm_campaign', 'Cost', 'Show', 'Link_cliks', 'Install', 'Reg',
      'Ftd', 'adset', 'geo', 'brand', 'page', 'soc', 'bm', 'acc', 'camp',
      'adset1', 'int', 'placement', 'gender', 'age', 'text', 'creo', 'app',
      'os', 'mb']]

  df1['int'] = df1['int'].astype(str)
  df1['age'] = df1['age'].astype(str)
  df1_sum = df1.groupby(['Utm_campaign'], as_index=False).sum().copy()

  result = pd.merge(df, df1_sum, how="left", on=["Utm_campaign"], suffixes=('', '_x'))
  # Замена NaN
  result.fillna(0, inplace=True)

  # Вычисление
  result['Cost'] = result['Cost'] - result['Cost_x']
  result['Show'] = result['Show'] - result['Show_x']
  result['Link_cliks'] = result['Link_cliks'] - result['Link_cliks_x']
  result['Install'] = result['Install'] - result['Install_x']
  result['Reg'] = result['Reg'] - result['Reg_x']
  result['Ftd'] = result['Ftd'] - result['Ftd_x']
  
  result.drop(['Cost_x', 'Show_x', 'Link_cliks_x', 'Install_x', 'Reg_x', 'Ftd_x'], axis=1, inplace=True)

  df_hourly = df1.append(result.loc[:, :])
  df_hourly_total = df_hourly.copy()
  
  # Добавление Status для фильтра
  df_stop['Status'] = 'Stop'
  df_hourly_total = pd.merge(df_hourly_total, df_stop, how="left", on=["bm"])  
  
  # Добавление Score для фильтра
  df_predict = predict[['Score', 'Utm_campaign']].copy()
  df_hourly_total = pd.merge(df_hourly_total, df_predict, how="left", on=["Utm_campaign"])



  # Группировка при выборке df_hourly + добавление ID
  df_group_hourly = pd.DataFrame(df_hourly_total.groupby(['bm'], as_index=False).sum().sort_values(['Cost'], ascending=False))
  df_group_hourly["ID"] = np.arange(0, 0 + len(df_group_hourly))
  df_group_hourly = df_group_hourly[['bm', 'ID']]

  df_hourly_total = pd.merge(df_hourly_total, df_group_hourly, how="left", on=["bm"])

  # Сводная таблица
  df_pivot = df_hourly_total.groupby(['ID', 'bm']).apply(lambda sub: sub.pivot_table(
      index=['Utm_campaign', 'Time'],
      values=['Cost', 'Install', 'Show', 'Link_cliks', 'Reg', 'Ftd'],
      aggfunc=np.sum,
      margins=True,
      margins_name='Total'
  ))

  df_pivot = df_hourly_total.groupby(['ID', 'bm']).apply(lambda sub: sub.pivot_table(
      index=['Utm_campaign', 'Time'],
      values=['Cost', 'Install', 'Show', 'Link_cliks', 'Reg', 'Ftd'],
      aggfunc=np.sum,
      margins=True,
      margins_name='Total'
  ))
        
  # Вычисление
  df_pivot = df_pivot.assign(Instal_cost = df_pivot.Cost/df_pivot.Install)
  df_pivot = df_pivot.assign(Reg_cost = df_pivot.Cost/df_pivot.Reg)
  df_pivot = df_pivot.assign(FTD_cost = df_pivot.Cost/df_pivot.Ftd)
  df_pivot = df_pivot.assign(CTR = df_pivot.Link_cliks/df_pivot.Show*100)

  # Замена Inf
  df_pivot.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
  # Два знака после запятой
  df_pivot = df_pivot.round(decimals=2)
  # Удалить колонки 
  df_pivot = df_pivot.drop(['Link_cliks', 'Show'], axis=1)

  # Сортировка по первому уровню
  df_pivot = df_pivot.sort_values(by = ["ID", 'Utm_campaign'], axis = 0, ascending=[True, False])

  df_pivot = df_pivot.droplevel('ID')
  df_hourly_total.drop(['ID', 'Score', 'Status'], axis=1, inplace=True)

  # Замена 0 на -
  df_pivot['Instal_cost'].replace(0, '-', inplace=True)
  df_pivot['Reg_cost'].replace(0, '-', inplace=True)
  df_pivot['FTD_cost'].replace(0, '-', inplace=True)

  df_pivot = df_pivot[['Cost', 'Install', 'Reg', 'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR']]
  print('1')
except:
  df_pivot = df.groupby(['bm', 'Utm_campaign', 'Time']).sum().sort_values(['Cost'], ascending=False)

  # Вычисление
  df_pivot = df_pivot.assign(Instal_cost = df_pivot.Cost/df_pivot.Install)
  df_pivot = df_pivot.assign(Reg_cost = df_pivot.Cost/df_pivot.Reg)
  df_pivot = df_pivot.assign(FTD_cost = df_pivot.Cost/df_pivot.Ftd)
  df_pivot = df_pivot.assign(CTR = df_pivot.Link_cliks/df_pivot.Show*100) 

  # Замена Inf
  df_pivot.replace([np.inf, -np.inf, np.nan], 0, inplace=True)
  # Два знака после запятой
  df_pivot = df_pivot.round(decimals=2)
  # Удалить колонки 
  df_pivot = df_pivot.drop(['Link_cliks', 'Show'], axis=1)
  
  # Замена 0 на -
  df_pivot['Instal_cost'].replace(0, '-', inplace=True)
  df_pivot['Reg_cost'].replace(0, '-', inplace=True)
  df_pivot['FTD_cost'].replace(0, '-', inplace=True)
  
  df_pivot = df_pivot[['Cost', 'Install', 'Reg', 'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR']]
  print('2')
finally:
  print("The 'try except' is finished")

2
The 'try except' is finished


### **Time page**

In [45]:
try:
  df_time = pd.pivot_table(df_hourly, values=['Cost','Link_cliks', 'Show', 'Install', 'Reg', 'Ftd'], 
                           index=['Time'], aggfunc='sum',
                           margins=True, margins_name='Total')

  # Вычисление
  df_time = df_time.assign(Instal_cost = df_time.Cost/df_time.Install)
  df_time = df_time.assign(Reg_cost = df_time.Cost/df_time.Reg)
  df_time = df_time.assign(FTD_cost = df_time.Cost/df_time.Ftd)
  df_time = df_time.assign(CTR = df_time.Link_cliks/df_time.Show*100)

  # Замена Inf
  df_time.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

  # Два знака после запятой
  df_time = df_time.round(decimals=2)

  # Удалить колонки 
  df_time = df_time.drop(['Link_cliks', 'Show'], axis=1)

  # Замена 0 на -
  df_time['Instal_cost'].replace(0, '-', inplace=True)
  df_time['Reg_cost'].replace(0, '-', inplace=True)
  df_time['FTD_cost'].replace(0, '-', inplace=True)
  df_time = df_time[['Cost', 'Install', 'Reg', 'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR']]
  print(1)
except:
  df_time = pd.DataFrame(df[['Time', 'Cost','Link_cliks', 'Show', 'Install', 'Reg', 'Ftd']])

  df_time = df_time.groupby('Time').sum().copy()

  # Вычисление
  df_time = df_time.assign(Instal_cost = df_time.Cost/df_time.Install)
  df_time = df_time.assign(Reg_cost = df_time.Cost/df_time.Reg)
  df_time = df_time.assign(FTD_cost = df_time.Cost/df_time.Ftd)
  df_time = df_time.assign(CTR = df_time.Link_cliks/df_time.Show*100)

  # Замена Inf
  df_time.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

  # Два знака после запятой
  df_time = df_time.round(decimals=2)

  # Удалить колонки 
  df_time = df_time.drop(['Link_cliks', 'Show'], axis=1)

  # Замена 0 на -
  df_time['Instal_cost'].replace(0, '-', inplace=True)
  df_time['Reg_cost'].replace(0, '-', inplace=True)
  df_time['FTD_cost'].replace(0, '-', inplace=True)

  df_time = df_time[['Cost', 'Install', 'Reg', 'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR']]
  print(2)
finally:
  print("The 'try except' is finished")



2
The 'try except' is finished


### **Creo page**

In [46]:
pivot_creo = pd.pivot_table(df, values=['Cost', 'Install', 'Show', 'Link_cliks', 'Reg', 'Ftd'], 
               index=['creo'], aggfunc='sum').sort_values('Cost', ascending=False)


# Вычисление
pivot_creo = pivot_creo.assign(Instal_cost = pivot_creo.Cost/pivot_creo.Install)
pivot_creo = pivot_creo.assign(Reg_cost = pivot_creo.Cost/pivot_creo.Reg)
pivot_creo = pivot_creo.assign(FTD_cost = pivot_creo.Cost/pivot_creo.Ftd)
pivot_creo = pivot_creo.assign(CTR = pivot_creo.Link_cliks/pivot_creo.Show*100)

# Замена Inf
pivot_creo.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

# Два знака после запятой
pivot_creo = pivot_creo.round(decimals=2)

# Удалить колонки 
pivot_creo = pivot_creo.drop(['Link_cliks', 'Show'], axis=1)

# Сортировка по первому уровню
# pivot_creo = pivot_creo.sort_values(by = ['creo', "app"], axis = 0, ascending=[True, True])

# Замена 0 на -
pivot_creo['Instal_cost'].replace(0, '-', inplace=True)
pivot_creo['Reg_cost'].replace(0, '-', inplace=True)
pivot_creo['FTD_cost'].replace(0, '-', inplace=True)

pivot_creo = pivot_creo[['Cost', 'Install', 'Reg', 'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR']]

### **App page**

In [47]:
pivot_app = pd.pivot_table(df, values=['Cost', 'Install', 'Show', 'Link_cliks', 'Reg', 'Ftd'], 
               index=['app'], aggfunc='sum').sort_values('Cost', ascending=False)

# Вычисление
pivot_app = pivot_app.assign(Instal_cost = pivot_app.Cost/pivot_app.Install)
pivot_app = pivot_app.assign(Reg_cost = pivot_app.Cost/pivot_app.Reg)
pivot_app = pivot_app.assign(FTD_cost = pivot_app.Cost/pivot_app.Ftd)
pivot_app = pivot_app.assign(CTR = pivot_app.Link_cliks/pivot_app.Show*100)

# Замена Inf
pivot_app.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

# Два знака после запятой
pivot_app = pivot_app.round(decimals=2)

# Удалить колонки 
pivot_app = pivot_app.drop(['Link_cliks', 'Show'], axis=1)

# Сортировка по первому уровню
# pivot_app = pivot_app.sort_values(by = ['app', "creo"], axis = 0, ascending=[True, False])

# Замена 0 на -
pivot_app['Instal_cost'].replace(0, '-', inplace=True)
pivot_app['Reg_cost'].replace(0, '-', inplace=True)
pivot_app['FTD_cost'].replace(0, '-', inplace=True)

pivot_app = pivot_app[['Cost', 'Install', 'Reg', 'Ftd', 'Instal_cost', 'Reg_cost', 'FTD_cost', 'CTR']]

### **Information page**

In [48]:
try:
  df_info = df_hourly
  print(1)
except:
  df_info = df[['Time', 'Utm_campaign', 'Cost', 'Show', 'Link_cliks', 
          'Install', 'Reg', 'Ftd', 'adset', 'geo', 'brand', 'page', 
          'soc', 'bm', 'acc', 'camp', 'adset1', 'int', 'placement', 
          'gender', 'age', 'text', 'creo', 'app', 'os', 'mb']]
  print("2")
finally:
  print("The 'try except' is finished")

2
The 'try except' is finished


## **Save file**

In [49]:
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
predict.to_excel(writer, sheet_name='Predict', index=False)
df_pivot.to_excel(writer, sheet_name='Total')
df_time.to_excel(writer, sheet_name='Time')
pivot_creo.to_excel(writer, sheet_name='Creo')
pivot_app.to_excel(writer, sheet_name='App')
df_info.to_excel(writer, sheet_name='Information', index=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet_predict = writer.sheets['Predict']
worksheet_total = writer.sheets['Total']
worksheet_time = writer.sheets['Time']
worksheet_creo = writer.sheets['Creo']
worksheet_app = writer.sheets['App']

# Number Format Categorie
currency_format = workbook.add_format({'num_format': 
                                       '_-* # ##0_-;-* # ##0_-;_-* "-"??_-;_-@_-'})
center = workbook.add_format({'align': 'center'})
# format2 = workbook.add_format({'num_format': '_-* # ##0,00\ _₽_-;-* # ##0,00\ _₽_-;_-* "-"??\ _₽_-;_-@_-'})

# Set the column width and format.
worksheet_predict.set_column('A:A', 15)
worksheet_predict.set_column('B:B', 50)
worksheet_predict.set_column('C:F', None, currency_format)
worksheet_predict.set_column('K:L', 12, center)

worksheet_total.set_column('A:A', 18)
worksheet_total.set_column('B:B', 50)
worksheet_total.set_column('C:C', None, center)
worksheet_total.set_column('D:G', None, currency_format)

worksheet_time.set_column('A:A', None, center)
worksheet_time.set_column('B:E', None, currency_format)
worksheet_time.set_column('F:I', 12)

worksheet_creo.set_column('A:A', 30)
worksheet_creo.set_column('B:E', None, currency_format)

worksheet_app.set_column('A:A', 30)
worksheet_app.set_column('B:E', None, currency_format)

# Close the Pandas Excel writer and output the Excel file.
writer.save()